# This model here is trained on 4 selected weather stations to predict one given weather station

## Loading data and converting to dataframe

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import datetime
import sklearn

### Solar intensity data

In [2]:
#Load initial data
hourly_sun_intensity = pd.read_excel('2-10_21_524-2 Andmed.xlsx', sheet_name = 'tunni sum.kiirgus', header = 1)

In [3]:
#Update column names by shortening them and converting to English
newColumnNames = dict()
newColumnNames["Aasta"] = "y"
newColumnNames["Kuu"] = "m"
newColumnNames["Päaev"] = "d"
newColumnNames["Kell (UTC)"] = "time"
for columnName in hourly_sun_intensity.columns:
    if "kiirgus" in columnName:
        newColumnNames[columnName] = "solar_"+columnName.replace(" summaarne kiirgus, W/m²", "")
#newColumnNames = ["y", "m", "d", "time"]+["solar_"+columnName.replace(" summaarne kiirgus, W/m²", "") for columnName in hourly_sun_intensity.columns if "kiirgus" in columnName]
hourly_sun_intensity = hourly_sun_intensity.rename(columns=newColumnNames)
#hourly_sun_intensity.columns = newColumnNames

In [4]:
# Some weather stations have changed locations over time, as the differences between their locations are rather small (less than 8 km)
# We at first do not make separation between them

def join_columns(c1, c2, nc, df, column_id): # Function for joining columns, where an area has two weather measuring points
    data = []
    cs = [c1, c2]
    for i, rows in df[cs].iterrows():
        if (pd.isna(rows[0]) == True) & (pd.isna(rows[1]) == False):
            data.append(round(rows[1], 2))
        elif (pd.isna(rows[0]) == False) & (pd.isna(rows[1]) == True):
            data.append(round(rows[0], 2))
        elif (pd.isna(rows[0]) == False) & (pd.isna(rows[1]) == False):
            data.append(round(rows.mean(), 2))
        elif (pd.isna(rows[0]) == True) & (pd.isna(rows[1]) == True):
            data.append(rows[0])

    df = df.drop(columns = [c1, c2])
    df.insert(column_id, nc, data)
    
    return df

In [5]:
#Merge columns, which are due to weather station moving
hourly_sun_intensity = join_columns('solar_Narva', 'solar_Narva-Jõesuu', 'solar_Narva', hourly_sun_intensity, 4)
hourly_sun_intensity = join_columns('solar_Pärnu-Sauga', 'solar_Pärnu', 'solar_Pärnu', hourly_sun_intensity, 5)

In [6]:
#Drop rows where some value is missing
hourly_sun_intensity = hourly_sun_intensity.dropna()
#If value is -1 it corresponds to night, set it to 0
hourly_sun_intensity = hourly_sun_intensity.replace(-1, 0)

In [7]:
#Shift the times -X minutes to facilitate predicting future solar intensity from existing
def shiftDateTime(df, numberOfHours):
    dateTimes = []
    for i in range(len(df)):
        row = df.iloc[i]
        dateTimes+=[datetime.datetime.combine(datetime.date(row.y, row.m, row.d), row.time)+datetime.timedelta(hours=numberOfHours)]
    df2 = copy.deepcopy(df)
    df2["y"] = [date.year for date in dateTimes]
    df2["m"] = [date.month for date in dateTimes]
    df2["d"] = [date.day for date in dateTimes]
    df2["time"] = [date.time() for date in dateTimes]
    
    return df2
    
    

In [8]:
hourly_sun_intensity_Shifted = shiftDateTime(hourly_sun_intensity, -1)

### Data from different weather stations

#### Locations


Tallinn-Harku
Laius: N 59°23´53´´
Pikkus: E 24°36´10´´
Decimal
Lat: 59.398055
Long: 24.602778


Haapsalu meteoroloogiajaam
Laius N 58°56´40´´
Pikkus E 23°33´18´´
Decimal
Lat: 58.944444
Long: 23.555

Narva
Laius: N 59°23´22´´
Pikkus: E 28°06´33´´
Decimal
Lat: 59.389444
Long: 28.109167

Pärnu
Laius: N 58°23´4,44´´
Pikkus: E 24°29´6,71´´
Decimal
Lat: 58.384556
Long: 24.485197

Roomassaare
Laius: N 58°13’05”
Pikkus: E 22°30’23”
Decimal
Lat: 58.218056 
Long: 22.506389 

Tartu-Tõravere meteoroloogiajaam
Laius: N 58°15´51´´
Pikkus: E 26°27´41´
Decimal
Lat: 58.264167
Long: 26.461389

Tiirikoja järvejaam
Laius: N 58°51´55´´
Pikkus: E 26°57´08´´
Decimal
Lat: 58.865278
Long: 26.952222

Vilsandi rannikujaam
Laius: N 58°22´58”
Pikkus: E 21°48´51”
Deciaml
Lat: 58.382778
Long: 21.814167

In [9]:
weather_station_coordinates = dict()
weather_station_coordinates["tallinn"] = [59.398055, 24.602778]
weather_station_coordinates["haapsalu"] = [58.944444, 23.555]
weather_station_coordinates["narva"] = [59.389444, 28.109167]
weather_station_coordinates["parnu"] = [59.389444, 28.109167]
weather_station_coordinates["roomassaare"] = [58.218056, 22.506389]
weather_station_coordinates["tartu"] = [58.264167, 26.461389]
weather_station_coordinates["tiirikoja"] = [58.865278, 26.952222]
weather_station_coordinates["vilsandi"] = [58.382778, 21.814167]

In [10]:
selectedColumns = ["Aasta", "Kuu", "Päev", "Kell (UTC)", "Õhutemperatuur °C","Suhteline õhuniiskus %", "10 minuti keskmine tuule kiirus m/s", "10 minuti keskmine tuule suund"]

In [11]:
#Update column names by shortening them and converting to English

def updateColumnNames(df, location):
    newColumnNames = dict()
    newColumnNames["Aasta"] = "y"
    newColumnNames["Kuu"] = "m"
    newColumnNames["Päev"] = "d"
    newColumnNames["Kell (UTC)"] = "time"
    newColumnNames["Õhutemperatuur °C"] = f"temp_{location}"
    newColumnNames["10 minuti keskmine tuule kiirus m/s"] = f"wind_speed_{location}"
    newColumnNames["Õhurõhk jaama kõrgusel hPa"] = f"pressure_{location}"
    newColumnNames["Suhteline õhuniiskus %"] = f"rel_humidity_{location}"
    newColumnNames["10 minuti keskmine tuule suund"] = f"wind_dir_{location}"
    df = df.rename(columns=newColumnNames)
    return df

In [12]:
def getFromXlsx(filename, columns, location):
    #Load xlsx
    df = pd.read_excel(filename, header = 1)
    df = df[columns]
    #Drop rows where data is missing
    #df = df.dropna()
    #Update column names for clarity
    df = updateColumnNames(df, location)
    return df

In [13]:
#Get Tallinn
data_tallinn = getFromXlsx("./data/Tallinn-Harku_2004-2020.xlsx", selectedColumns, "tallinn")
#data_tallinn["lat_tallinn"] = len(data_tallinn)*[weather_station_coordinates["tallinn"][0]]
#data_tallinn["long_tallinn"] = len(data_tallinn)*[weather_station_coordinates["tallinn"][1]]
#Get Roomassaare
data_roomassaare = getFromXlsx("./data/Roomassaare_2008-2020.xlsx", selectedColumns, "roomassaare")
#data_roomassaare["lat_roomassaare"] = len(data_roomassaare)*[weather_station_coordinates["roomassaare"][0]]
#data_roomassaare["long_roomassaare"] = len(data_roomassaare)*[weather_station_coordinates["roomassaare"][1]]
#Merge tables
data_weather = data_tallinn.merge(data_roomassaare, how='left', on=["y", "m", "d", "time"])

In [14]:
#Get Vilsandi
data_vilsandi = getFromXlsx("./data/Vilsandi_2004-2020.xlsx", selectedColumns, "vilsandi")
#data_vilsandi["lat_vilsandi"] = len(data_vilsandi)*[weather_station_coordinates["vilsandi"][0]]
#data_vilsandi["long_vilsandi"] = len(data_vilsandi)*[weather_station_coordinates["vilsandi"][1]]
#Merge tables
data_weather = data_weather.merge(data_vilsandi, how='left', on=["y", "m", "d", "time"])

In [15]:
##Get Parnu
#data_parnu1 = getFromXlsx("./data/Parnu-Sauga_01.12.2004-31.03.2019.xlsx", selectedColumns, "parnu")
#data_parnu2 = getFromXlsx("./data/Parnu_01.04.2019-2020.xlsx", selectedColumns, "parnu")
#data_parnu = data_parnu1.append(data_parnu2)

##Merge tables
#data_weather = data_weather.merge(data_parnu, how='left', on=["y", "m", "d", "time"])

In [16]:
data_Tartu = getFromXlsx("./data/Tartu-Toravere_2004-2020.xlsx", selectedColumns, "tartu")
#data_Tartu["lat_tartu"] = len(data_Tartu)*[weather_station_coordinates["tartu"][0]]
#data_Tartu["long_tartu"] = len(data_Tartu)*[weather_station_coordinates["tartu"][1]]
#Merge tables

data_weather = data_weather.merge(data_Tartu, how='left', on=["y", "m", "d", "time"])


In [17]:
data_Narva = getFromXlsx("./data/Narva_19.12.2013-2020.xlsx", selectedColumns, "narva")
data_weather = data_weather.merge(data_Narva, how='left', on=["y", "m", "d", "time"])

In [18]:
#Once again drop all rows where some row is missing
data_weather = data_weather.dropna()

### Join weather and solar data

In [19]:
data_solar_weather = hourly_sun_intensity_Shifted.merge(data_weather, how='left', on=["y", "m", "d", "time"])

In [20]:
hours = []
for i in range(len(data_solar_weather)):
    hours+=[data_solar_weather.iloc[i].time.hour]
data_solar_weather["h"] = hours


In [21]:
data_solar_weather = data_solar_weather.dropna()

In [22]:
data_solar_weather.columns

Index(['y', 'm', 'd', 'time', 'solar_Narva', 'solar_Pärnu', 'solar_Haapsalu',
       'solar_Tallinn-Harku', 'solar_Roomassaare', 'solar_Tartu-Tõravere',
       'solar_Tiirikoja', 'solar_Vilsandi', 'temp_tallinn',
       'rel_humidity_tallinn', 'wind_speed_tallinn', 'wind_dir_tallinn',
       'temp_roomassaare', 'rel_humidity_roomassaare',
       'wind_speed_roomassaare', 'wind_dir_roomassaare', 'temp_vilsandi',
       'rel_humidity_vilsandi', 'wind_speed_vilsandi', 'wind_dir_vilsandi',
       'temp_tartu', 'rel_humidity_tartu', 'wind_speed_tartu',
       'wind_dir_tartu', 'temp_narva', 'rel_humidity_narva',
       'wind_speed_narva', 'wind_dir_narva', 'h'],
      dtype='object')

# Let's train models

## Baseline, predict Tallinn using Tallinn data

In [98]:
X = data_solar_weather[['m','d', 'h', 'temp_tallinn',
       'rel_humidity_tallinn', 'wind_speed_tallinn', 'wind_dir_tallinn']]

y = data_solar_weather[['solar_Tallinn-Harku']]

In [99]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [100]:
from sklearn.tree import DecisionTreeRegressor
dtr= DecisionTreeRegressor(max_depth = 80)
dtr.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=80)

In [101]:
from sklearn.metrics import mean_squared_error
# squared = False returns RMSE, otherwise MSE
mean_squared_error(y_test, dtr.predict(X_test), squared = False)

85.12577897000247

In [102]:
results_df = pd.DataFrame()
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


60.01107241412734

In [103]:
rf= RandomForestRegressor()
rf.fit(X_train_scaled, y_train)
mean_squared_error(y_test, rf.predict(X_test_scaled), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


59.84158218583287

## Baseline 2, predict Pärnu using Talinn data

In [108]:
X = data_solar_weather[['m','d', 'h', 'temp_tallinn',
       'rel_humidity_tallinn', 'wind_speed_tallinn', 'wind_dir_tallinn']]

y = data_solar_weather[['solar_Pärnu']]

In [109]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [110]:
from sklearn.tree import DecisionTreeRegressor
dtr= DecisionTreeRegressor(max_depth = 80)
dtr.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=80)

In [111]:
from sklearn.metrics import mean_squared_error
# squared = False returns RMSE, otherwise MSE
mean_squared_error(y_test, dtr.predict(X_test), squared = False)

103.23589459344848

In [112]:
results_df = pd.DataFrame()
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


72.36021142195774

In [113]:
rf= RandomForestRegressor()
rf.fit(X_train_scaled, y_train)
mean_squared_error(y_test, rf.predict(X_test_scaled), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


71.9543162945638

## 4 other stations, predict Pärnu, different parameters

4 Stations Tartu, Tallinn, Roomassaare, Vilsandi, predict Pärnu
dtr 96.26316991627043
rf 66.69709972042308


### Data Prep

In [75]:
data_solar_weather.columns

Index(['y', 'm', 'd', 'time', 'solar_Narva', 'solar_Pärnu', 'solar_Haapsalu',
       'solar_Tallinn-Harku', 'solar_Roomassaare', 'solar_Tartu-Tõravere',
       'solar_Tiirikoja', 'solar_Vilsandi', 'temp_tallinn',
       'rel_humidity_tallinn', 'wind_speed_tallinn', 'wind_dir_tallinn',
       'temp_roomassaare', 'rel_humidity_roomassaare',
       'wind_speed_roomassaare', 'wind_dir_roomassaare', 'temp_vilsandi',
       'rel_humidity_vilsandi', 'wind_speed_vilsandi', 'wind_dir_vilsandi',
       'temp_tartu', 'rel_humidity_tartu', 'wind_speed_tartu',
       'wind_dir_tartu', 'temp_narva', 'rel_humidity_narva',
       'wind_speed_narva', 'wind_dir_narva', 'h'],
      dtype='object')

In [76]:
X = data_solar_weather[['m','d', 'h', 'temp_tallinn',
       'rel_humidity_tallinn', 'wind_speed_tallinn', 'wind_dir_tallinn',
       'temp_roomassaare', 'rel_humidity_roomassaare',
       'wind_speed_roomassaare', 'wind_dir_roomassaare', 'temp_vilsandi',
       'rel_humidity_vilsandi', 'wind_speed_vilsandi', 'wind_dir_vilsandi',
       'temp_tartu', 'rel_humidity_tartu', 'wind_speed_tartu',
       'wind_dir_tartu']]

y = data_solar_weather[['solar_Pärnu']]

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

In [78]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Decision Tree

In [83]:
from sklearn.tree import DecisionTreeRegressor
dtr= DecisionTreeRegressor(max_depth = 80)
dtr.fit(X_train, y_train)

In [84]:
from sklearn.metrics import mean_squared_error
# squared = False returns RMSE, otherwise MSE
mean_squared_error(y_test, dtr.predict(X_test), squared = False)

DecisionTreeRegressor(max_depth=80)


5 Stations Tartu, Tallinn, Roomassaare, Vilsandi, Narva, predict Pärnu
dtr 
rf

In [86]:
dtr.fit(X_train_scaled, y_train)
mean_squared_error(y_test, dtr.predict(X_test_scaled), squared = False)

100.30923244650441

### Random Forest

In [87]:
results_df = pd.DataFrame()
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test), squared = False)

In [90]:
rf= RandomForestRegressor()
rf.fit(X_train_scaled, y_train)
mean_squared_error(y_test, rf.predict(X_test_scaled), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


66.99407415307772

In [41]:
############### Random Forest ##############################

n_estimatorss = [100]
max_depths=[20]
min_samples_splits=[4,6,8,16]
seeds = [1]
results_df = pd.DataFrame(columns=['model',"seed","n_estimators","max_depth","min_samples_split",'TrainError', 'ValError', 'deltaErrors'])
for seed in seeds:
    for n_estimator in n_estimatorss:
        for max_d in max_depths:
            for min_ss in min_samples_splits:
                rf = RandomForestRegressor(criterion="squared_error", n_estimators=n_estimator, max_depth=max_d, min_samples_split=min_ss, random_state=seed).fit(X_train, y_train)
                trainError = mean_squared_error(y_train, rf.predict(X_train), squared = False)
                valError = mean_squared_error(y_test, rf.predict(X_test), squared = False)
                results_df = results_df.append({'model': 'RF',"seed":seed,"n_estimators":n_estimator,"max_depth":max_d,"min_samples_split":min_ss,"TrainError":trainError, 'ValError':valError, 'deltaErrors':abs(trainError-valError)}, ignore_index=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if sys.path[0] == '':


In [42]:
results_df.sort_values("ValError")

,model,seed,n_estimators,max_depth,min_samples_split,TrainError,ValError,deltaErrors
0,RF,1,100,20,4,30.873757,67.119425,36.245668
1,RF,1,100,20,6,33.273907,67.285888,34.011981
2,RF,1,100,20,8,35.777680,67.457766,31.680086
3,RF,1,100,20,16,44.046657,68.225880,24.179223


In [40]:
results_df.sort_values("deltaErrors")

,model,seed,n_estimators,max_depth,min_samples_split,TrainError,ValError,deltaErrors
0,RF,1,200,20,4,30.61484,66.889728,36.274888


In [61]:
results_df.sort_values("deltaErrors")

,model,seed,n_estimators,max_depth,min_samples_split,TrainError,ValError,deltaErrors
5,RF,1,100,40,8,36.957563,74.915639,37.958076
8,RF,1,100,50,8,36.956893,74.917349,37.960456
14,RF,1,100,70,8,36.956893,74.917349,37.960456
11,RF,1,100,60,8,36.956893,74.917349,37.960456
17,RF,1,100,80,8,36.956893,74.917349,37.960456
...,...,...,...,...,...,...,...,...
33,RF,1,200,80,4,29.717919,74.207728,44.489809
30,RF,1,200,70,4,29.717919,74.207728,44.489809
27,RF,1,200,60,4,29.717919,74.207728,44.489809
24,RF,1,200,50,4,29.717919,74.207728,44.489809


In [91]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor

gbr = HistGradientBoostingRegressor(max_iter=100, max_depth=10)
gbr.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:17: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  "Since version 1.0, "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


HistGradientBoostingRegressor(max_depth=10)

## Predict Pärnu, use 4 stations, use only one type of variables

## Temperature

In [115]:
X = data_solar_weather[['m','d', 'h', 'temp_tallinn',
       'temp_roomassaare', 'temp_vilsandi',
       'temp_tartu']]

y = data_solar_weather[['solar_Pärnu']]

In [116]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

In [117]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [118]:
from sklearn.tree import DecisionTreeRegressor
dtr= DecisionTreeRegressor(max_depth = 80)
dtr.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=80)

In [119]:
dtr.fit(X_train_scaled, y_train)
mean_squared_error(y_test, dtr.predict(X_test_scaled), squared = False)

107.92307200732515

In [120]:
results_df = pd.DataFrame()
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


77.96527929643392

In [121]:
rf= RandomForestRegressor()
rf.fit(X_train_scaled, y_train)
mean_squared_error(y_test, rf.predict(X_test_scaled), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


77.86844172490507

## Humidity

In [123]:
X = data_solar_weather[['m','d', 'h',
       'rel_humidity_tallinn', 'rel_humidity_roomassaare',
       'rel_humidity_vilsandi', 'rel_humidity_tartu']]

y = data_solar_weather[['solar_Pärnu']]

In [124]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)

In [125]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [126]:
from sklearn.tree import DecisionTreeRegressor
dtr= DecisionTreeRegressor(max_depth = 80)
dtr.fit(X_train, y_train)
dtr.fit(X_train_scaled, y_train)
mean_squared_error(y_test, dtr.predict(X_test_scaled), squared = False)

103.89287296065847

In [127]:
results_df = pd.DataFrame()
from sklearn.ensemble import RandomForestRegressor
rf= RandomForestRegressor()
rf.fit(X_train, y_train)
mean_squared_error(y_test, rf.predict(X_test), squared = False)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


72.07701172527304

### Idea of trying Stacking to see if it works better

In [128]:
from sklearn.ensemble import StackingRegressor

In [ ]:
estimators = [
    ('lr', RidgeCV()),
    ('svr', LinearSVR(random_state=42))
]

In [24]:
data_solar_weather

,y,m,d,time,solar_Narva,solar_Pärnu,solar_Haapsalu,solar_Tallinn-Harku,solar_Roomassaare,solar_Tartu-Tõravere,...,wind_dir_vilsandi,temp_tartu,rel_humidity_tartu,wind_speed_tartu,wind_dir_tartu,temp_narva,rel_humidity_narva,wind_speed_narva,wind_dir_narva,h
30116,2013,12,20,13:00:00,0.0,1.0,0.0,3,2.0,3,...,242.0,2.2,97.0,2.3,210.0,2.0,95.0,5.7,231.0,13
30117,2013,12,20,14:00:00,0.0,0.0,0.0,0,0.0,0,...,212.0,2.7,96.0,2.8,229.0,2.0,95.0,3.4,216.0,14
30118,2013,12,20,15:00:00,0.0,0.0,0.0,0,0.0,0,...,203.0,2.8,95.0,2.2,217.0,2.7,92.0,3.3,209.0,15
30119,2013,12,20,16:00:00,0.0,0.0,0.0,0,0.0,0,...,209.0,3.0,94.0,3.0,212.0,2.1,95.0,4.5,214.0,16
30120,2013,12,20,17:00:00,0.0,0.0,0.0,0,0.0,0,...,220.0,3.1,94.0,2.9,203.0,2.1,92.0,6.4,220.0,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88783,2020,12,31,18:00:00,0.0,0.0,0.0,0,0.0,0,...,175.0,1.1,97.0,1.5,210.0,0.8,99.0,2.7,195.0,18
88784,2020,12,31,19:00:00,0.0,0.0,0.0,0,0.0,0,...,181.0,1.1,93.0,1.1,229.0,0.7,98.0,2.9,191.0,19
88785,2020,12,31,20:00:00,0.0,0.0,0.0,0,0.0,0,...,189.0,1.0,96.0,1.5,224.0,0.8,98.0,2.7,202.0,20
88786,2020,12,31,21:00:00,0.0,0.0,0.0,0,0.0,0,...,189.0,0.8,95.0,1.4,202.0,0.8,98.0,2.8,200.0,21


In [ ]:
## Feature selection for one model 

# Defining forward selection algorithm 

from sklearn.tree import DecisionTreeRegressor
from sklearn import model_selection
from sklearn.metrics import mean_squared_error

def forward_selector(maxFeatureN, train_y, test_y, train_X, test_X, random_seed, cv_k, originalData, 
                     isPandas = False):
    
    np.random.seed(random_seed)
    
    returnList = {"maxFeatureN": maxFeatureN}
    
    candFeatures = list(range(train_X.shape[1]))
    selection = []
    
    for i in range(maxFeatureN):
        errScores = []
        
        for c in candFeatures: 
            S = list(selection)
            S.append(c)
            
            if isPandas: 
                train_X_S = train_X.iloc[:, S]
            else:
                train_X_S = train_X[:,S]

            # fitting the model
            dtr= DecisionTreeRegressor(max_depth = 80)
            err = np.mean(model_selection.cross_val_score(dtr, 
                                                                train_X_S, 
                                                                train_y, 
                                                                cv=cv_k, 
                                                                ))

            errScores.append(err)
    
        bestIndex = np.argmin(errScores)
        
        #Update current best selection
        selection.append(candFeatures[bestIndex])
        del candFeatures[bestIndex]
    
    if isPandas:
        train_X_S = train_X.iloc[:,selection]
    else:
        train_X_S = train_X[:,selection]
    #print(train_X_S.shape)
    if isPandas:
        test_X_S = test_X.iloc[:,selection]
    else:
        test_X_S = test_X[:,selection]
        
    #print(test_X_S.shape)


    #svmClassifier = sklearn.svm.LinearSVC()
    dtr= DecisionTreeRegressor(max_depth = 80)
    dtr.fit(train_X_S, train_y)
    
    train_y_hat = dtr.predict(train_X_S)
    test_y_hat = dtr.predict(test_X_S)
    
    returnList["featuresSelectedNumeric"] = [selection]
    returnList["trainErr"] = mean_squared_error(train_y, train_y_hat, squared = False)
    returnList["testErr"] = mean_squared_error(test_y, test_y_hat, squared = False)

    
    return returnList
    
    
X = data_solar_weather[['m','d', 'h', 'temp_tallinn',
       'rel_humidity_tallinn', 'wind_speed_tallinn', 'wind_dir_tallinn',
       'temp_roomassaare', 'rel_humidity_roomassaare',
       'wind_speed_roomassaare', 'wind_dir_roomassaare', 'temp_vilsandi',
       'rel_humidity_vilsandi', 'wind_speed_vilsandi', 'wind_dir_vilsandi',
       'temp_tartu', 'rel_humidity_tartu', 'wind_speed_tartu',
       'wind_dir_tartu']]

y = data_solar_weather[['solar_Pärnu']]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=111)



## Running the algorithm with stopping rules

errResults = []
for ii in (range (1,10,1)):
    
    result = forward_selector(ii, y_train, y_test, X_train_scaled, X_test_scaled, 1, 10, X)
    errResults.append(result["trainErr"])
    print("For cardinality", ii, "Best features are: ", result["featuresSelectedNumeric"])
    print("Giving performance in train set of ", result["trainErr"])
    print("Giving performance in test set of ", result["testErr"])
    print("\n")

    
## Finding best stes for different stopping rules

## Test untill you see decline in performance

best_untill_decline_cardinality = None
for i in range(len(errResults)):
    if i == len(errResults)-1:
        break
    elif errResults[i] > errResults[i+1]:
        next
    elif errResults[i] <= errResults[i+1]:
        best_untill_decline = i+1
        break
        
        
    
print("Best set up to a point where model performance on train data starts to decrease is at cardinality: ", 
      best_untill_decline)


print("Best overall set is: ", (np.argmax(errResults)+1) )